# Install Packages

In [2]:
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from typing import Union

# Quick Start Guide for BOM Analysis

## Step 1: Get Data
This would probably be a sql query converted to a dataframe. But, for the purpose of this notebook, I will manually create a simple BOM

In [3]:
test_df = pd.DataFrame([
    {'PARENTPARTNUM':'Complete','PARTNUM': 'A','TYPECODE': 'M','BOMLEVEL':0,'LEADTIME':8},
    {'PARENTPARTNUM':'A','PARTNUM': 'B','TYPECODE': 'M','BOMLEVEL':1,'LEADTIME':10},
    {'PARENTPARTNUM':'A','PARTNUM': 'C','TYPECODE': 'P','BOMLEVEL':1,'LEADTIME':38},
    {'PARENTPARTNUM':'A','PARTNUM': 'D','TYPECODE': 'M','BOMLEVEL':1,'LEADTIME':7},
    {'PARENTPARTNUM':'B','PARTNUM': 'E','TYPECODE': 'M','BOMLEVEL':2,'LEADTIME':9},
    {'PARENTPARTNUM':'B','PARTNUM': 'F','TYPECODE': 'P','BOMLEVEL':2,'LEADTIME':21},
    {'PARENTPARTNUM':'B','PARTNUM': 'G','TYPECODE': 'M','BOMLEVEL':2,'LEADTIME':6},
    {'PARENTPARTNUM':'D','PARTNUM': 'H','TYPECODE': 'P','BOMLEVEL':2,'LEADTIME':23},
    {'PARENTPARTNUM':'D','PARTNUM': 'I','TYPECODE': 'M','BOMLEVEL':2,'LEADTIME':10},
    {'PARENTPARTNUM':'D','PARTNUM': 'J','TYPECODE': 'P','BOMLEVEL':2,'LEADTIME':45},
    {'PARENTPARTNUM':'E','PARTNUM': 'K','TYPECODE': 'P','BOMLEVEL':3,'LEADTIME':25},
    {'PARENTPARTNUM':'E','PARTNUM': 'L','TYPECODE': 'P','BOMLEVEL':3,'LEADTIME':27},
    {'PARENTPARTNUM':'G','PARTNUM': 'M','TYPECODE': 'P','BOMLEVEL':3,'LEADTIME':41},
    {'PARENTPARTNUM':'G','PARTNUM': 'N','TYPECODE': 'P','BOMLEVEL':3,'LEADTIME':35},
    {'PARENTPARTNUM':'I','PARTNUM': 'O','TYPECODE': 'P','BOMLEVEL':3,'LEADTIME':24},
    {'PARENTPARTNUM':'I','PARTNUM': 'P','TYPECODE': 'M','BOMLEVEL':3,'LEADTIME':4},
    {'PARENTPARTNUM':'P','PARTNUM': 'Q','TYPECODE': 'P','BOMLEVEL':4,'LEADTIME':21},
    {'PARENTPARTNUM':'P','PARTNUM': 'R','TYPECODE': 'P','BOMLEVEL':4,'LEADTIME':25},
    {'PARENTPARTNUM':'P','PARTNUM': 'S','TYPECODE': 'P','BOMLEVEL':4,'LEADTIME':29}
])

## Step 2: Create the BomAnalyzer class

In [4]:
class BomAnalyzer:
    def __init__(self):
        """
        Initialize the BOM Analyzer class.
        """
        
        self.bom_df = None
        self.root = None
        self.loaded = False
        self.DG = None
        self.purchased_parts = None
        self.manufactured_parts = None

    def load(self, bom_df: pd.DataFrame, root: str):
        """
        Load the BOM Analyzer with a dataframe containing BOM data.

        Args:
        bom_df: pd.DataFrame
            Expected columns:
            - PARENTPARTNUM
            - PARTNUMTYPECODE
            - BOMLEVEL
            - LEADTIME

        root: str = root part of the BOM
        """

        # re-init values
        self.bom_df = bom_df
        self.root = root
        self.purchased_parts = self.bom_df[self.bom_df['TYPECODE']=='P']['PARTNUM'].drop_duplicates().values
        self.manufactured_parts = self.bom_df[self.bom_df['TYPECODE']=='M']['PARTNUM'].drop_duplicates().values
        self.DG = nx.DiGraph()

        # load graph
        part_nodes = self.bom_df['PARTNUM'].drop_duplicates().to_list()
        for part in part_nodes:
            self.DG.add_node(part)
        
        for _, row in self.bom_df.iterrows():
            parent, child, bom_level, part_type, lead_time = row['PARENTPARTNUM'], row['PARTNUM'], row['BOMLEVEL'], row['TYPECODE'], row['LEADTIME']
            self.DG.add_edge(parent, child, lead_time=lead_time, bom_level=bom_level, part_type=part_type)

        # change loaded state
        self.loaded = True

        print('BOM has been successfully loaded')


    def get_longest_leadtime_path(self):
        """
        Get a dataframe of the longest leadtime sequence.

        Returns:
        - pd.DataFrame
        
        """
        
        if not self.loaded:
            print('BOM not loaded')
            return
            
        longest_path = nx.dag_longest_path(self.DG, weight='lead_time')
        consecutive_pairs = set(zip(longest_path[1:], longest_path))
        longest_path_df = self.bom_df[self.bom_df[['PARTNUM', 'PARENTPARTNUM']].apply(tuple, axis=1).isin(consecutive_pairs)].sort_values(by='BOMLEVEL').drop_duplicates()
        return longest_path_df

    
    def get_longest_leadtime_value(self):
        """
        Get the value of the longest leadtime sequence.

        Returns:
        - float
        """
        if not self.loaded:
            print('BOM not loaded')
            return
            
        return nx.dag_longest_path_length(self.DG, weight='lead_time')


    def get_all_paths(self):
        """
        Get all paths between root part and purchased 
        """
        if not self.loaded:
            print('BOM not loaded')
            return
            
        all_rows = []
        for part in self.purchased_parts:
            paths = nx.all_simple_paths(self.DG,self.root,part)
            for path in paths:
                total_length = 0
                for i in range(len(path)-1):
                    source, target = path[i], path[i+1]
                    edge = self.DG[source][target]
                    length = edge['lead_time']
                    total_length += length
                # print('{}: {}'.format(path, total_length))
                row = {
                    'path': path,
                    'bomlevel': len(path),
                    'leadtime': total_length
                }
            
                all_rows.append(row)
        
        all_paths = pd.DataFrame(all_rows).sort_values(by='leadtime', ascending=False).reset_index(drop=True)
        
        return all_paths


## Step 3: Load the BOM

In [7]:
# init
test_bom = BomAnalyzer()
test_bom.load(test_df, root='A')

BOM has been successfully loaded


## Step 4: Get BOM Insights

### Attributes from the BOM class

In [8]:
# test attributes
print(f'bom_df:\n{test_bom.bom_df}')
print(f'root: {test_bom.root}')
print(f'loaded: {test_bom.loaded}')
print(f'DG: {test_bom.DG}')
print(f'manufactured_parts: {test_bom.manufactured_parts}')
print(f'purchased_parts: {test_bom.purchased_parts}')

bom_df:
   PARENTPARTNUM PARTNUM TYPECODE  BOMLEVEL  LEADTIME
0       Complete       A        M         0         8
1              A       B        M         1        10
2              A       C        P         1        38
3              A       D        M         1         7
4              B       E        M         2         9
5              B       F        P         2        21
6              B       G        M         2         6
7              D       H        P         2        23
8              D       I        M         2        10
9              D       J        P         2        45
10             E       K        P         3        25
11             E       L        P         3        27
12             G       M        P         3        41
13             G       N        P         3        35
14             I       O        P         3        24
15             I       P        M         3         4
16             P       Q        P         4        21
17             P    

### Modules from BOM class

Get all BOM paths from root to purchased part (aka leaf)

In [9]:
test_bom.get_all_paths()

,path,bomlevel,leadtime
0,"[A, B, G, M]",4,57
1,"[A, D, J]",3,52
2,"[A, B, G, N]",4,51
3,"[A, D, I, P, S]",5,50
4,"[A, D, I, P, R]",5,46
5,"[A, B, E, L]",4,46
6,"[A, B, E, K]",4,44
7,"[A, D, I, P, Q]",5,42
8,"[A, D, I, O]",4,41
9,"[A, C]",2,38


Get the longest path. This is effectively where the BOM if bottlenecked during a cold start

In [10]:
test_bom.get_longest_leadtime_path()

,PARENTPARTNUM,PARTNUM,TYPECODE,BOMLEVEL,LEADTIME
0,Complete,A,M,0,8
1,A,B,M,1,10
6,B,G,M,2,6
12,G,M,P,3,41


In [12]:
longest_leadtime_days = test_bom.get_longest_leadtime_value()
print(f'Maximum leadtime is {longest_leadtime_days} days.')

Maximum leadtime is 65 days.
